In [2]:
!apt install -q -y libspatialindex-dev libproj-dev proj-data proj-bin libgeos-dev
!npm install -g mapshaper

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
m##################) ⠏ reify:mproj: http fetch GET 200 https://registry.npmjs.oegistr
added 67 packages, and audited 68 packages in 7s

6 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


In [38]:
import os
import sys
from pathlib import Path
import subprocess
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from IPython.display import display, Markdown

In [33]:
WORKING_DIR = Path(os.getcwd())
DATA = Path(f'{WORKING_DIR.parents[0]}/datasets/HE/gadm_eez_original_merged')
dataPath  = list(DATA.glob('*.shp'))[0]

In [34]:
data_gpd = gpd.read_file(dataPath)

In [32]:
data_gpd.head()

,GID_0,Shape_Leng,Shape_Area,geometry
0,ABW,0.963634,0.015131,"POLYGON ((-69.97820 12.46986, -69.97847 12.469..."
1,AFG,57.103371,62.749594,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,73.796528,103.818655,"MULTIPOLYGON (((11.73347 -16.67255, 11.73347 -..."
3,AIA,1.318321,0.007116,"MULTIPOLYGON (((-63.42375 18.58903, -63.42375 ..."
4,ALA,42.232199,0.243769,"MULTIPOLYGON (((21.32195 59.74986, 21.32195 59..."


In [36]:
def interior_to_polygon(geom: Polygon):
  """
  Takes a Shapely geometry and returns interiors as Shapely geometry.
  """
  types = [geom.geom_type] if type(geom.geom_type) == str else geom.geom_type.tolist()
  # check geometry is a polygon 
  if types and 'Polygon' in types:
    # check it contains a ring
    ring = geom.is_ring
    if ring.any() and geom.interiors.to_list()[0]:
      # extract shapely LinearRing(s)
      rings_list = [Polygon(g) for g in geom.interiors.to_list()[0]]
      out = MultiPolygon(rings_list) if len(rings_list) > 1 else rings_list[0]
    else:
      print("No interior ring")
      out = "NA"
  else:
    print("Requires a Polygon")
    out = "NA"     
  return out

def exterior_to_polygon(geom):
  """
  Takes a Shapely geometry and returns exterior ring as Shapely geometry.
  """
  types = geom.geom_type.tolist() or []
  # check geometry is a polygon 
  if types and 'Polygon' in types:
    # check it contains a ring
    ring = geom.is_ring
    if ring.any() and geom.exterior.tolist()[0]:
      # extract shapely LinearRing
      out = Polygon(geom.exterior.tolist()[0])
    else:
      print("No exterior ring")
      out = "NA"
  else:
    print("Requires a Polygon")
    out = "NA"     
  return out

In [39]:
def execute_command(command):
  """
  Executes a command in the shell and returns the output.
  """
  try:
    out = subprocess.check_output(command, shell=True)
  except subprocess.CalledProcessError as e:
    out = e.output
  return out

In [41]:
o = f"{dataPath}_clean.shp"
execute_command(f'mapshaper-xl 16 -i snap {dataPath} \
        -clean gap-fill-area=30km2 \
        -dissolve GID_0 \
        -o format=shapefile {o} force')

Allocating 8 GB of heap memory
[i] Snapped 103967 points
